<a href="https://colab.research.google.com/github/manjitullal/unsupervised/blob/main/ContentBased_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unrar

In [ ]:
path = '/content/drive/MyDrive/datasets/movies/movie_data_merge.rar'

In [ ]:
!unrar x $path

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_pickle('/content/movie_data_merge')
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,cast,crew,score
314,False,25000000,"Drama, Crime",278.0,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,51.6454,"Castle Rock Entertainment, Warner Bros",United States of America,1994-09-23,2.834147e+07,142.0,English,Released,The Shawshank Redemption,8.5,8358.0,"[{'cast_id': 3, 'character': 'Andy Dufresne', ...","[{'credit_id': '52fe4231c3a36847f800b127', 'de...",8.445870
837,False,6000000,"Drama, Crime",238.0,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",41.1093,"Paramount Pictures, Alfran Productions",United States of America,1972-03-14,2.450664e+08,175.0,"English, Italiano, Latin",Released,The Godfather,8.5,6024.0,"[{'cast_id': 5, 'character': 'Don Vito Corleon...","[{'credit_id': '52fe422bc3a36847f80093db', 'de...",8.425439
10345,False,13200000,"Comedy, Drama, Romance",19404.0,tt0112870,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",34.457,Yash Raj Films,India,1995-10-20,1.000000e+08,190.0,हिन,Released,Dilwale Dulhania Le Jayenge,9.1,661.0,"[{'cast_id': 1, 'character': 'Raj Malhotra', '...","[{'credit_id': '57a3054a9251417c57000d7a', 'de...",8.421458
12525,False,185000000,"Drama, Action, Crime, Thriller",155.0,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167,"DC Comics, Legendary Pictures, Warner Bros, D...","United Kingdom, United States of America",2008-07-16,1.004558e+09,152.0,"English, 普通话",Released,The Dark Knight,8.3,12269.0,"[{'cast_id': 35, 'character': 'Bruce Wayne / B...","[{'credit_id': '55a0eb4a925141296b0010f8', 'de...",8.265477
2854,False,63000000,Drama,550.0,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.8696,"Twentieth Century Fox Film Corporation, Regen...","Germany, United States of America",1999-10-15,1.008538e+08,139.0,English,Released,Fight Club,8.3,9678.0,"[{'cast_id': 4, 'character': 'The Narrator', '...","[{'credit_id': '55731b8192514111610027d7', 'de...",8.256385


In [ ]:
# select random 10000 indices 

random_indices = np.random.choice(data.shape[0], size=10000, replace=False)
small_data = data.loc[random_indices]
small_data.reset_index(drop=True, inplace=True)
small_data.shape

(10000, 22)

In [ ]:
small_data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,cast,crew,score
0,False,0,"Horror, Thriller",27324.0,tt0286751,ja,回路,"After one of their friends commits suicide, st...",4.12232,"Kadokawa Pictures, Nippon Television Network ...",Japan,2001-02-10,0.0,118.0,日本語,Released,Pulse,6.1,89.0,"[{'cast_id': 18, 'character': 'Ryosuke Kawashi...","[{'credit_id': '52fe4541c3a368484e04fc75', 'de...",5.790430
1,False,2700000,Drama,352205.0,tt3043252,hi,Parched,"In a rural Indian village, four ordinary women...",3.322681,"Blue Waters Motion Pictures, Shivalaya Entert...","India, United Kingdom, United States of America",2015-11-14,0.0,116.0,"English, हिन",Released,Parched,8.5,23.0,"[{'cast_id': 4, 'character': 'Lajjo', 'credit_...","[{'credit_id': '55b9364292514123dd0029ae', 'de...",5.980422
2,False,0,"Comedy, Foreign, Romance",24681.0,tt1213929,en,White Wedding,"The loyal, committed and very decent Elvis (Ke...",0.35292,,South Africa,2009-01-01,0.0,93.0,"Afrikaans, , isiZulu, English",Released,White Wedding,2.5,2.0,"[{'cast_id': 1000, 'character': 'Tony', 'credi...","[{'credit_id': '52fe44a3c3a368484e02e733', 'de...",5.579735
3,False,0,"Comedy, Music",43257.0,tt0049034,en,Bundle of Joy,"Kitschy musical remake of ""Bachelor Mother"". D...",0.441986,Edmund Grainger Productions,United States of America,1956-12-12,0.0,98.0,English,Released,Bundle of Joy,7.0,5.0,"[{'cast_id': 2, 'character': 'Dan Merlin', 'cr...","[{'credit_id': '52fe4639c3a36847f80f33f3', 'de...",5.660104
4,False,16800000,"Drama, Mystery, Romance",18615.0,tt0422295,en,Fur: An Imaginary Portrait of Diane Arbus,In 1958 New York Diane Arbus is a housewife an...,5.10832,"River Road Entertainment, Edward R, Iron Film",United States of America,2006-08-30,2281089.0,122.0,English,Released,Fur: An Imaginary Portrait of Diane Arbus,5.8,73.0,"[{'cast_id': 5, 'character': 'Diane Arbus', 'c...","[{'credit_id': '564c55309251417c46005e72', 'de...",5.675181


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            stop_words = 'english')

small_data['overview'] = small_data['overview'].fillna('')

In [ ]:
tfv_matrix = tfv.fit_transform(small_data['overview'])
tfv_matrix.shape

(10000, 12481)

In [ ]:
from sklearn import preprocessing

similarity_matrix = np.dot(preprocessing.normalize(tfv_matrix, axis=1), preprocessing.normalize(tfv_matrix, axis=1).T)
similarity_matrix = similarity_matrix.toarray()

In [ ]:
indices = pd.Series(small_data.index, index=small_data['original_title']).drop_duplicates()

In [ ]:
small_data.columns

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'cast', 'crew', 'score'],
      dtype='object')

In [ ]:
small_data[['adult','budget','genres','original_title','revenue','vote_count','score']].loc[20:,].head()

,adult,budget,genres,original_title,revenue,vote_count,score
20,False,6500000,"Action, Adventure, Drama, Family",The Adventures of Huck Finn,0.0,43.0,5.741464
21,False,0,"Horror, Thriller",The Cradle,0.0,2.0,5.599489
22,False,1000000,"Horror, Adventure, Thriller",The Snake King,0.0,7.0,5.525252
23,False,0,"Comedy, Drama",Bank Holiday,0.0,3.0,5.625259
24,False,0,Comedy,Her Crowning Glory,0.0,4.0,5.634861


In [ ]:
import pprint

movie_overview = small_data[small_data.original_title == 'The Irish Pub'].overview.tolist()
pprint.pprint(movie_overview, depth=1)

['The tradition of the Irish pub. Speaking to pub owners all over Ireland, '
 'Alex Fegan gets into the heart of what makes "the Irish pub" the institution '
 'that it is.']


In [ ]:
def recommendation(title, similarity_matrix):
    idx = indices[title]
    sig_scores = list(enumerate(similarity_matrix[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return small_data[['adult','budget','genres','original_title','revenue','vote_count','score']].iloc[movie_indices]

In [ ]:
recommendation('The Irish Pub', similarity_matrix)

,adult,budget,genres,original_title,revenue,vote_count,score
7007,False,0,Comedy,Last Call,0.0,6.0,5.548898
944,False,0,"Drama, Action, Crime",Ash Wednesday,0.0,14.0,5.608719
9432,False,0,Drama,Last Night At The Alamo,0.0,2.0,5.598254
1060,False,0,"Comedy, Romance",The Matchmaker,0.0,18.0,5.666950
2529,False,28000000,"Drama, Thriller",Michael Collins,16900000.0,98.0,6.067121
613,False,0,Comedy,A Night Out,0.0,10.0,5.611277
3085,False,580000,Documentary,Miesten vuoro,0.0,11.0,5.700685
4638,False,0,"Drama, Thriller, War",I See a Dark Stranger,0.0,6.0,5.599501
7414,False,0,"Documentary, Music",Oil City Confidential,0.0,2.0,5.653810
6430,False,0,"Crime, Mystery, Thriller",La tête d'un homme,0.0,1.0,5.614392
